In [4]:
from threading import Thread
from multiprocessing import Process
import multiprocessing
import subprocess
import signal
import time
import os
import sys

In [ ]:
def start_stress(num_cores, time):
    print("Stressing ",num_cores," cores for ",time," seconds")
    t= str(time)+'s'
    cmd= 'stress '+'-c '+str(num_cores)+' -t '+ t
    pid_stress = subprocess.Popen(['stress', '-c', str(num_cores), "-t", t],
                        stdout=subprocess.PIPE, stderr=subprocess.PIPE, preexec_fn=os.setsid)
    return pid_stress

In [ ]:
def add_data(f):
    global str_Cpu
    global str_Wt
    global str_Ct
    global str_Ac
    global str_Pr
    global str_Re
    global str_Line
    
    str_Line= "\n"+str_Line+"|\n"
    f.write(str_Cpu)
    f.write(str_Line)
    f.write(str_Wt)
    f.write(str_Line)
    f.write(str_Ct)
    f.write(str_Line)
    f.write(str_Ac)
    f.write(str_Line)
    f.write(str_Pr)
    f.write(str_Line)
    f.write(str_Re)
    f.write(str_Line)

In [ ]:
def get_data(m_name, output, times):
    global num_cores
    global list_cpus
    global name_dataset
    global str_Cpu
    global str_Wt
    global str_Ct
    global str_Ac
    global str_Pr
    global str_Re
    global str_Line
    
    f= open(name_dataset+'_results.txt', 'a+')
    
    output= output.decode("utf-8")
    times= times.decode("utf-8")
    
    if(num_cores== list_cpus[0]):
        m_name= "\n"+m_name+"\n "
        f.write(m_name)
        str_Cpu= "           |    Idle    |"
        str_Wt= "| Wall time |"
        str_Ct= "| Cpu time  |"
        str_Ac= "| Accuracy  |"
        str_Pr= "| Precision |"
        str_Re= "| Recall    |"
        str_Line="|-----------|------------"
        
    elif(num_cores > 0):
        str_Cpu= str_Cpu+" "+str(num_cores)+"  stress  |"
        str_Line= str_Line+"|------------"
    
    lines= times.split("\n")
    
    wall_time= lines[1].replace("real\t","").split("m")
    minutes= str(wall_time[0]+"m")
    sec= float(wall_time[1].replace("s",""))
    sec= str(round(sec, 2))+"s "
    
    str_wallTime= " "+minutes+sec
    if(len(str_wallTime) < 11):
        str_wallTime=str_wallTime+" "*(11-len(str_wallTime))
        
    str_Wt= str_Wt+" "+str_wallTime+"|"
    
    user_time= lines[2].split("\t")[1]
    user_time= user_time.split("m")
    
    sys_time= lines[3].split("\t")[1]
    sys_time= sys_time.split("m")
    
    minutes= int(user_time[0])+int(sys_time[0])
    minutes= str(minutes)+"m"
    sec= float(user_time[1].replace("s",""))+float(sys_time[1].replace("s",""))
    sec= str(round(sec,2))+"s "
    
    str_cpuTime= " "+minutes+sec
    
    if(len(str_cpuTime) < 11):
        str_cpuTime=str_cpuTime+" "*(11-len(str_cpuTime))  
    
    str_Ct= str_Ct+" "+str_cpuTime+"|"
    
    output=output.split("Accuracy")[1]#para quitar caracteres extranos
    output=output.split("\n")
    
    ac_output= " "+output[0].split(":")[1]
    pre_output= " "+output[1].split(":")[1]
    rec_output= " "+output[2].split(":")[1]
    
    if(len(ac_output) < 11):
        ac_output= ac_output+" "*(11-len(ac_output))
    if(len(pre_output) < 11):
        pre_output= pre_output+" "*(11-len(pre_output))    
    if(len(rec_output) < 11):
        rec_output= rec_output+" "*(11-len(rec_output))    
    
    str_Ac= str_Ac+ac_output+" |"
    str_Pr= str_Pr+pre_output+" |"
    str_Re= str_Re+rec_output+" |"
    
    if(num_cores==list_cpus[-1]):
        add_data(f)
    #f.write(output)
    #f.write("\n------------------------------\n")
    
    #print("OUTPUT: ", output)
    #print("TIMES: ", times)

In [ ]:
def model(model_num):
    global dataSet
    global num_cores
    global name_dataset
    
        
    m_name= ""
    
    if model_num == 0:
        m_name= "Logistic Regression"
        print("Executing ",m_name)
        output, times = subprocess.Popen(['bash', '-c', 'time "$@" 2>&1', '_',"ipython", './Modelos/lRegression.ipynb', dataSet],
                        stdout=subprocess.PIPE, stderr=subprocess.PIPE).communicate()
        
    if model_num == 1:
        m_name= "Maquinas de soporte vectorial"
        print("Executing ",m_name)
        m_name= "\n"+m_name
        output, times = subprocess.Popen(['bash', '-c', 'time "$@" 2>&1', '_',"ipython", './Modelos/svm.ipynb', dataSet],
                        stdout=subprocess.PIPE, stderr=subprocess.PIPE).communicate()

    if model_num == 2:
        m_name= "Gradient Boosting"
        print("Executing ",m_name)
        output, times = subprocess.Popen(['bash', '-c', 'time "$@" 2>&1', '_',"ipython", './Modelos/gBoosting.ipynb', dataSet],
                        stdout=subprocess.PIPE, stderr=subprocess.PIPE).communicate()
    
    if model_num == 3:
        m_name= "Random Forest"
        print("Executing ",m_name)
        output, times = subprocess.Popen(['bash', '-c', 'time "$@" 2>&1', '_',"ipython", './Modelos/rForest_pc.ipynb', dataSet],
                        stdout=subprocess.PIPE, stderr=subprocess.PIPE).communicate()
        
        
    get_data(m_name, output,times)

In [5]:
if (len(sys.argv) != 2):
    error=""
    error=error+"Error: wrong number of arguments, execute as: \npython "+sys.argv[0]+" dataset_name"
    sys.exit(error)

SystemExit: Error: wrong number of arguments, execute as: 
python /home/nuriadj/anaconda3/envs/tfg/lib/python3.9/site-packages/ipykernel_launcher.py dataset_name

/home/nuriadj/anaconda3/envs/tfg/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3465: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
#Hacer una version que ejecute de seguido los modelos solo (sin stress)
t= 200
list_cpus= [0,2,4,8]
dataSet= sys.argv[1]
name_dataset= "Pc_"+dataSet.split("/")[-1].split(".")[0]
f= open(name_dataset+'_results.txt', 'w+')
f.close()

str_Cpu= ""
str_Wt= ""
str_Ct= ""
str_Ac= ""
str_Pr= ""
str_Re= ""
str_Line= ""

for i in range(4):
    for j in list_cpus:
        if(j > 0):
            num_cores= j
            pid_stress= start_stress(j, t)
        else:
            num_cores= j
            print("\nCpu Idle")
            
        model(i)
        
        if j > 0:
            print("Waiting stress to end...")
            os.killpg(os.getpgid(pid_stress.pid), signal.SIGKILL)
            #pid_stress.communicate()
            #pid_stress.wait()
        
        print("Sleeping for 5 sec...\n")
        time.sleep(5)